## Walk-through: modelling impact of climate hazards on cement plants

In this walk-through, assets of type 'CementPlant' which belong to the 'ManufacturingAsset' class are modelled. This is a worked example of how to specify the vulnerability of an arbitrary asset to different climate hazards. A custom vulnerability configuration is defined in csv format, both for acute and chronic hazards.

In [1]:
# pip install nbformat pandas plotly requests
from IPython.display import display
import json
from typing import NamedTuple
from dependency_injector import providers
from physrisk.container import Container
from physrisk.vulnerability_models.vulnerability import VulnerabilityModelsFactory
from physrisk.vulnerability_models.config_based_impact_curves import (
    config_items_from_csv,
    config_items_to_df,
)
import plotly.io

plotly.io.renderers.default = "notebook"

Define a portfolio. The ```asset_class``` must match one of the pre-defined ```physrisk``` classes, but ```type``` is free text that links to the vulnerability functions. In this example we use 'CementPlant' for the ```type```. The field can be hierarchical using a '\\' separator, but here for simplicity there is just a single level.

In [2]:
portfolio = {
    "items": [
        {
            "asset_class": "ManufacturingAsset",
            "type": "CementPlant",
            "latitude": 46.73753446,
            "longitude": 15.57335874,
        },
        {
            "asset_class": "ManufacturingAsset",
            "type": "CementPlant",
            "latitude": 251.06066169,
            "longitude": -115.1701174,
        },
    ]
}
request = {
    "assets": portfolio,
    "include_asset_level": True,
    "include_calc_details": True,
    "include_measures": False,
    "years": [2050],
    "scenarios": ["historical", "ssp585"],
}

The configuration for the CementPlants is contained in the file ```cement_plants_example_config.csv```. We will load into a Pandas DataFrame for ease of viewing.

In [3]:
config_items = config_items_from_csv("cement_plants_example_config.csv")
config_df = config_items_to_df(config_items)
display(config_df.head())

,hazard_class,asset_class,asset_identifier,indicator_id,indicator_units,impact_id,impact_units,curve_type,points_x,points_y,points_z,points_kind,cap_of_points_x,cap_of_points_y,activation_of_points_x
0,Wind,Asset,"type=Generic,location=Asia",max_speed,m/s,damage,None,indicator/piecewise_linear,"[20.0, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0, 60....","[0.0, 0.0, 0.001, 0.007, 0.024, 0.058, 0.109, ...",None,None,None,None,None
1,Wind,Asset,"type=Generic,location=China Mainland",max_speed,m/s,damage,None,indicator/piecewise_linear,"[20.0, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0, 60....","[0.0, 0.0, 0.0, 0.002, 0.008, 0.02, 0.039, 0.1...",None,None,None,None,None
2,Wind,Asset,"type=Generic,location=Europe",max_speed,m/s,damage,None,indicator/piecewise_linear,"[20.0, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0, 60....","[0.0, 0.0, 0.0, 0.003, 0.012, 0.029, 0.057, 0....",None,None,None,None,None
3,Wind,Asset,"type=Generic,location=Generic",max_speed,m/s,damage,None,indicator/piecewise_linear,"[20.0, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0, 60....","[0.0, 0.0, 0.0, 0.003, 0.012, 0.029, 0.057, 0....",None,None,None,None,None
4,Wind,Asset,"type=Generic,location=North America",max_speed,m/s,damage,None,indicator/piecewise_linear,"[20.0, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0, 60....","[0.0, 0.0, 0.0, 0.003, 0.012, 0.029, 0.057, 0....",None,None,None,None,None


Having defined the configuration the calculation is run. Dependency injection is used to generate vulnerability models from the configuration. 

In [4]:
# we make use of dependency injection to inject a vulnerability model that accepts the configuration.
container = Container()
container.override_providers(
    vulnerability_models_factory=providers.Factory(
        VulnerabilityModelsFactory, config=config_items
    )
)
requester = container.requester()
response = json.loads(
    requester.get(request_id="get_asset_impact", request_dict=request)
)

The impacts of chronic and acute hazards are generally treated differently in financial calculations. We assume that chronic hazards such as heat and drought already affect the revenue generation (attributable to a given asset). For example:

- A construction company that requires high-intensity outdoor labour from some employees is already affected by lower productivity (than some ideal) for a certain portion of the year as a result of conditions of high temperature and humidity. Typically of interest is the *incremental* impact on productivity. 
- A power generating company may already factor into their operations a lower contribution from power generating assets under conditions of drought, but it may be relevant to calculate how that contribution could decrease over time.

For chronic hazards, the metric of interest is therefore likely to be the *change* in revenue generation (attributable to an asset) between current climate and a potential future climate.

In contrast we assume that acute hazards do not already affect the revenue generation, in particular infrequent but severe events. Whereas insurance premiums may affect company financials, a 1-in-100 year event does not impact revenue generation in the same way as an annual occurrence. 

In [5]:
asset_index = 0
asset_impacts = response["asset_impacts"][asset_index]["impacts"]


# for convenience unpack using a key
class Key(NamedTuple):
    hazard_type: str
    scenario_id: str
    year: str


asset_impact_dict = {}
for i in asset_impacts:
    key = i["key"]
    asset_impact_dict[Key(key["hazard_type"], key["scenario_id"], key["year"])] = i

hazard_types = set(k.hazard_type for k in asset_impact_dict.keys())

for hazard_type in hazard_types:
    impact_histo = asset_impact_dict[Key(hazard_type, "historical", "None")]
    impact_ssp585 = asset_impact_dict[Key(hazard_type, "ssp585", "2050")]
    mean_histo = impact_histo["impact_mean"]
    mean_ssp585 = impact_ssp585["impact_mean"]
    if hazard_type in ["ChronicHeat", "Drought"]:
        print(
            f"For hazard {hazard_type}, modelled reduction to revenue generation is {(mean_ssp585 - mean_histo) * 100:.2g}%."
        )
    else:
        print(
            f"For hazard {hazard_type}, modelled Annual Average Loss is {mean_histo * 100:.2g}% (current) and {mean_ssp585 * 100:.2g}% (future climate)."
        )

For hazard RiverineInundation, modelled Annual Average Loss is 0% (current) and 0% (future climate).
For hazard Drought, modelled reduction to revenue generation is 0.35%.
For hazard Wind, modelled Annual Average Loss is 0% (current) and 0% (future climate).
For hazard ChronicHeat, modelled reduction to revenue generation is 0.82%.
For hazard CoastalInundation, modelled Annual Average Loss is 0% (current) and 0% (future climate).
